Extract and preprocess text data for each language

In [ ]:
# def clean_sentences(content):
#     # Remove opening html tag + article title
#     trimmed_open = re.sub(r'<doc\b[^>]*>\s*(?:.*\n)?', '', content)
#     # Remove closing html tag
#     trimmed_close = re.sub(r'<\/doc\b[^>]*>', '', trimmed_open)
#     # Remove single-word subheaders
#     trimmed_sub = re.sub(r'(?<=\n)\w+\.(?=\n)', '', trimmed_close)
#     # Split at periods while trying to overlook abbreviations
#     return re.split(r'(?<!\w\.\w)(?<!\w\.)(?<!Sr)(?<!Mr)(?<!Ms)(?<!Mrs)(?<!Jr)(?<!Dr)\.(?!\w)', trimmed_sub)

In [ ]:
### Quick test
# with open('/Users/k/Docs/School/Tuebingen/Thesis/iscl-thesis/WikiExtractor/lfn_wiki.txt', 'r', encoding='utf-8') as f:
#     text = f.read()
# cleaned = clean_sentences(text)
# alpha = re.compile(r'[^a-z]')
# with open('/Users/k/Docs/School/Tuebingen/Thesis/iscl-thesis/lfn_example.txt', 'w', encoding='utf-8') as out:
#     for sentence in cleaned:
#         s = ' '.join([re.sub(alpha, '', word.strip().lower()) for word in sentence.split()])
#         s = re.sub(r'\s+', ' ', s).strip()
#         if len(s.split()) > 1:
#             out.write(s + '.' + '\n')

In [ ]:
# # Directory with extracted files
# maindir = '/Users/k/Docs/School/Tuebingen/Thesis/Corpuses/HtmlWiki'
# wikidirs = [sub for sub in os.listdir(maindir) if sub != '.DS_Store']
# # Dict of languages + their corresponding alphabets
# alphabets = {
#     'lfn': re.compile(r'[^a-z]'), 
#     'ia': re.compile(r'[^a-z]'), 
#     'eo': 'abcĉdefgĝhĥijĵklmnoprsŝtuŭvz', 
#     'de': 'abcdefghijklmnopqrstuvwxyzäöüß', 
#     'fr': 'abcdefghijklmnopqrstuvwxyzéàèùëüïâêîôûçæœ', 
#     'en': re.compile(r'[^a-z]')
#     }

# # Estimated smallest wc (from lfn corpus)
# min_wc = 1265000

# for wiki in wikidirs:
#     wiki_fullpath = os.path.join(maindir, wiki)
#     lang = os.path.splitext(os.path.basename(wiki))[0].split('_')[0]
#     current_wc = 0
#     if lang in alphabets:
#         alpha = alphabets[lang]
#     with open(f'/Users/k/Docs/School/Tuebingen/Thesis/iscl-thesis/current_corpora/{lang}_wiki_html.txt', 'w', encoding='utf-8') as out:
#         for root, dirs, files in os.walk(wiki_fullpath):
#             for file in files:
#                 file_path = os.path.join(root, file)
#                 # Make sure only corpora files 
#                 if '/wiki' in file_path:
#                     with open(file_path) as f: 
#                         content = f.read()
#                         f.close()
#                     sentences = clean_sentences(content)
#                     # Check if alpha in dict is regex
#                     if isinstance(alpha, re.Pattern):
#                         for sentence in sentences:
#                             # Remove all chars not language's alphabet
#                             s = ' '.join([re.sub(alpha, '', word.strip().lower()) for word in sentence.split()])
#                             # Remove leftover whitespaces
#                             s = re.sub(r'\s+', ' ', s).strip()
#                             if (len(s.split()) + current_wc) < min_wc:
#                                 current_wc += len(s.split())
#                                 out.write(s + '.' + '\n')
#                             else:
#                                 break
#                     else:
#                         for sentence in sentences:
#                             # Remove all chars not language's alphabet
#                             s = ' '.join([word.lower() for word in sentence.split() if all(letter in alpha for letter in word.lower())])
#                             # Remove leftover whitespaces
#                             s = re.sub(r'\s+', ' ', s).strip()
#                             if (len(s.split()) + current_wc) < min_wc:
#                                 current_wc += len(s.split())
#                                 out.write(s + '.' + '\n')
#                             else:
#                                 break
#     out.close()
#     print(f'{lang}_wiki_html.txt finished')

Using /iscl-thesis/WikiExtractor

In [1]:
import re, os, string

def clean_sentences(content):
    # Remove page titles
    no_titles = re.sub(r'(?<=\s\n).+:\n', '', content)
    # Remove section heaeders
    no_headers = re.sub(r'.+\/\/\/\/\/\n', '', no_titles)
    # Remove picture links
    no_pics = re.sub(r'(?<=\n)(\w+\|)+.+[^\.]\n', '', no_headers)
    # Remove fragments
    no_fragments = re.sub(r'(?<=\n)\w+\:\w+.*[^\.](?=\n)', '', no_pics)
    # Remove other links
    no_links = re.sub(r'http[^\s\n]+(?=\s|\n)', '', no_fragments)
    # Remove paranthesis
    no_paranthesis = re.sub(r'\([^\)]+\)', '', no_links)
    # Removes braces
    no_braces = re.sub(r'\[+\w+[^\]]*\]+', '', no_paranthesis)
    # Remove lists
    no_lists = re.sub(r'(?<=\n)[^\.\n]+\:\s?\n', '', no_braces)
    # Remove html tags
    no_html = re.sub(r'\<[^\>]+\>', '', no_lists)
    # Remove number fragments
    no_num = re.sub(r'\d+(?=\n)', '', no_html)
    # Split at periods while trying to overlook abbreviations
    sentences = re.split(r'(?<!\w\.\w)(?<!\w\.)(?<!Sr)(?<!Mr)(?<!Ms)(?<!Mrs)(?<!Jr)(?<!Dr)\.(?!\w)', no_num)
    # Remove punctuation, numbers, make lowercase
    return [s.translate(str.maketrans('', '', string.punctuation + string.digits)).lower() for s in sentences]

In [2]:
# Directory with extracted files
maindir = '/Users/k/Docs/School/Tuebingen/Thesis/Corpuses/WikiExtractorCorpora/'
# Dict of languages + their corresponding alphabets
alphabets = {
    # Lingua Franca Nova
    'lfn': 'abcdefghijklmnopqrstuvwxyz', 
    # Interlingua
    'ia': 'abcdefghijklmnopqrstuvwxyz', 
    # Esperanto
    'eo': 'abcĉdefgĝhĥijĵklmnoprsŝtuŭvz', 
    # German
    'de': 'abcdefghijklmnopqrstuvwxyzäöüß', 
    # French
    'fr': 'abcdefghijklmnopqrstuvwxyzéàèùëüïâêîôûçæœ', 
    # English
    'en': 'abcdefghijklmnopqrstuvwxyz',
    # Finnish
    'fi': 'abcdefghijklmnopqrstuvwxyzšžåäö',
    # Tagalog
    'tl': 'abcdefghijklmnopqrstuvwxyzñ',
    # Turkish
    'tr': 'abcçdefgğhıijklmnoöprsştuüvyz',
    # Vietnamese
    'vi': 'aáàảãạăắằẳẵặâấầẩẫậbcdđeéèẻẽẹêếềểễệfghiíìỉĩịjklmnoóòỏõọôốồổỗộơớờởỡợpqrstuúùủũụưứừửữựvwxyýỳỷỹỵz',
    # Polish
    'pl': 'aąbcćdeęfghijklłmnńoópqrsśtuvwxyzźż',
    # Indonesian
    'id': 'abcdefghijklmnopqrstuvwxyz',
    # Ido
    'io': 'abcdefghijklmnopqrstuvwxyz',
    # Italian
    'it': 'abcdefghijklmnopqrstuvwxyz',
    # Dutch
    'nl': 'abcdefghijklmnopqrstuvwxyz',
    # Occitan
    'oc': 'abcdefghijklmnopqrstuvwxyzàèòáéíóúïüç',
    # Danish
    'da': 'abcdefghijklmnopqrstuvwxyzæøå',
    # Swedish
    'sv': 'abcdefghijklmnopqrstuvwxyzåäö',
    # Hungarian
    'hu': 'aábccsddzdzseéfggyhiíjkllymnnyoóöőpqrsszttyuúüűvwxyzzs',
    # Spanish
    'es': 'abcdefghijklmnopqrstuvwxyzñ',
    # Afrikaans
    'af': 'aáäbcdeéèêëfghiíîïjklmnŉoóôöpqrstuúûüvwxyýz',
    # Icelandic
    'is': 'aábdðeéfghiíjklmnoóprstuúvxyýþæö',
    }

# LFN corpus is the smallest size based on word count, at about 650000 words, so shrink all other corpora to about the same
max_wc = 650000

for file in os.listdir(maindir):
    if file.endswith('.txt'):
        fullpath = os.path.join(maindir, file)
        lang = os.path.splitext(os.path.basename(file))[0].split('_')[0]
        current_wc = 0
        if lang in alphabets:
            alpha = set(alphabets[lang])
        with open(f'/Users/k/Docs/School/Tuebingen/Thesis/iscl-thesis/current_corpora/{lang}_wiki_extractor.txt', 'w', encoding='utf-8') as out:
            with open(fullpath, 'r', encoding='utf-8') as f: 
                content = f.read()
                f.close()
            sentences = clean_sentences(content)
            for sentence in sentences:
                # Remove all chars not language's alphabet
                s = ''.join(char if char in alpha or char.isspace() else '' for char in sentence)
                # Remove leftover whitespaces
                s = re.sub(r'\s+', ' ', s).strip()
                # Remove sentence fragments containing only 1 word
                if len(s.split()) > 1:
                    if (len(s.split()) + current_wc) < max_wc:
                        current_wc += len(s.split())
                        out.write(s + '\n')
                    else:
                        break
                else:
                    continue
        
        print(f'{lang} corpus finished')
        out.close()

tl corpus finished
oc corpus finished
af corpus finished
eo corpus finished
en corpus finished
tr corpus finished
io corpus finished
de corpus finished
fr corpus finished
id corpus finished
vi corpus finished
sv corpus finished
ia corpus finished
nl corpus finished
es corpus finished
da corpus finished
it corpus finished
pl corpus finished
lfn corpus finished
fi corpus finished
is corpus finished
hu corpus finished
